In [1]:
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from tensorflow.keras import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from skimage import io, transform,img_as_float
from skimage.io import imread
from tensorflow.keras.models import load_model
import numpy as np
import cv2
from sklearn.metrics import label_ranking_average_precision_score
import time
%matplotlib inline


## Get data

In [ ]:
PATH = '/home/user/20191.DL/Proyect/data/'

img_ids = next(os.walk(PATH))[2]

data = []

for n, id_ in enumerate(img_ids):
    path = PATH + id_
    img = imread(path)
    img = img_as_float(img)
   
    data.append(img)
    


In [ ]:
data_np = np.array(data)
print data_np.shape


In [6]:
print(len(img_ids))

500


In [ ]:
aux = 0
while(aux < 20):
    pos = np.random.randint(0, 500)
    plt.imshow(data_np[pos])
    plt.show()
    aux += 1
   

In [ ]:
shape_data = len(data_np)
shape_img = data_np

print'Shape images', shape_img

In [ ]:
print np.amax(data_np)   

## Split data

In [ ]:
x_train, x_test = train_test_split(data_np, test_size=0.30, random_state=42)



In [ ]:
print x_train.dtype
x_train = x_train.astype(np.float64)


In [ ]:
x_train_norm = x_train / 255
x_test_norm = x_test / 255


In [ ]:
print np.amin(x_train_norm)

In [ ]:
x_train_norm_r = np.reshape(x_train_norm, (len(x_train_norm), 400, 400, 3))  # adapt this if using `channels_first` image data format
#x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format


In [ ]:
plt.imshow(x_train[1])

In [ ]:
x_test = x_test.astype('float32') / 255

## Create noisy


In [ ]:
datan = data[0]
datan = datan.astype('float32') / 255.
w = np.random.randint()
h=40
bx,by = np.random.randint(img.shape[1]-w), np.random.randint(img.shape[0]-h)
bx,by=80,80

bx,by = 100,50

value =datan[by:by+h,bx:bx+w]  + 0.5 * np.random.normal(loc=0.0, scale=1.0, size=(40,30,3)) 
datan[by:by+h,bx:bx+w] =value

noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [ ]:
def train_model():
    input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same', name='encoder')(x)

    # at this point the representation is (4, 4, 8) i.e. 128-dimensional

    x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

    autoencoder.fit(x_train_noisy, x_train,
                    epochs=20,
                    batch_size=128,
                    shuffle=True,
                    validation_data=(x_test_noisy, x_test),
                    callbacks=[TensorBoard(log_dir='/tmp/tb', histogram_freq=0, write_graph=False)])

    autoencoder.save('autoencoder.h5')

train_model()

In [ ]:
print('Loading mnist dataset')
t0 = time.time()
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

noise_factor = 0.3
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)
t1 = time.time()
print('mnist dataset loaded in: ', t1-t0)

print('Loading model :')
t0 = time.time()
# Load previously trained autoencoder
autoencoder = load_model('autoencoder.h5')
t1 = time.time()
print('Model loaded in: ', t1-t0)

In [ ]:
def plot_denoised_images():
    denoised_images = autoencoder.predict(x_test_noisy.reshape(x_test_noisy.shape[0], x_test_noisy.shape[1], x_test_noisy.shape[2],1))
    test_img = x_test_noisy[0]
   # plt.figure(figsize=(4,2))
    plt.figure(figsize=(20,5))
    plt.subplot(311)
    plt.imshow(test_img.reshape(28,28), cmap=plt.cm.Greys_r)
    plt.axis("off")
    #resized_test_img = cv2.resize(test_img, (280, 280))
    #cv2.imshow('input', resized_test_img)
    #cv2.waitKey(0)
    output = denoised_images[0]
    #plt.figure(figsize=(4,2))
    R= test_img[0]-denoised_images[0] 
    plt.subplot(312)
    plt.imshow( output.reshape(28,28), cmap=plt.cm.Greys_r)
    plt.axis("off")
    
    plt.subplot(313)
    plt.imshow( R.reshape(28,28), cmap=plt.cm.Greys_r)
    plt.axis("off")
    
    #resized_output = cv2.resize(output, (280, 280))
    #cv2.imshow('output', resized_output)
    #cv2.waitKey(0)

In [ ]:
plot_denoised_images()